In [121]:
import pandas as pd
import requests
import re

    
def LoadTXTnSplit(url,regx='\|| \| |--|==',repl=[["\n\n","|"],["\n"," "]]):
    r = requests.get(url)
    T=r.text
    for ri in repl:
        T=T.replace(ri[0],ri[1])
    return pd.Series(re.split(regx,T))
    #T.replace("\n\n","|").replace("\n"," "))

K=LoadTXTnSplit("https://raw.githubusercontent.com/tobiasseidel/koalitionsvertrag2018/master/koalitionsvertrag2018.md")
K=K[K!=""]
S=LoadTXTnSplit("https://raw.githubusercontent.com/nicmer/analysis-zeitonline/master/electoralprograms/spd.txt")
U=LoadTXTnSplit("https://raw.githubusercontent.com/nicmer/analysis-zeitonline/master/electoralprograms/union.txt")

# K =?= ( S + U ) / 2 ?

# Vergleich von Absätzen

In [122]:
#derived from https://stackoverflow.com/questions/15173225/calculate-cosine-similarity-given-2-sentence-strings
import math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

def similarity(txt1,txt2):
    vector1 = text_to_vector(txt1)
    vector2 = text_to_vector(txt2)
    return get_cosine(vector1, vector2)


In [137]:
def lookupWahlprogramm(W,content,returnValue=False):
    similarities=W.map(lambda x: similarity(x,content))
    if returnValue:
        return similarities.max()
    else:
        return similarities.idxmax()

KS=K.map(lambda x:lookupWahlprogramm(S,x))
sKS=K.map(lambda x:lookupWahlprogramm(S,x,True))
KU=K.map(lambda x:lookupWahlprogramm(U,x))
sKU=K.map(lambda x:lookupWahlprogramm(U,x,True))

# Zusammenstellung

In [143]:
Vergleich=pd.DataFrame({'K': K,   #Inhalt des Koalitionsvertrages
                        'KS' : KS,#index im Wahlprogramm der SPD
                        'KU': KU, #index im Wahlprogramm der Union
                        'S': list(S[KS]), # Inhalt des SPD-Wahlprogramms
                        'U': list(U[KU]), # Inhalt des Union-Wahprogramms
                        'yS':KS*max(K.index)/KS.max(), # y position SPD
                        'yU':KU*max(K.index)/KU.max(), # yposition Union
                        'sKS':sKS, #Ähnlichkeit SPD
                        'sKU':sKU})#Ähnlichkeit Union
Vergleich[10:15]

,K,KS,KU,S,U,sKS,sKU,yS,yU
15,Den sozialen Zusammenhalt in unserem Land woll...,361,53,"Wir wollen, dass mehr Güter über die Schiene u...",- Wir haben in dieser Wahlperiode unsere Inves...,0.610895,0.638609,1713.960641,630.003650
16,"Wir wollen, dass die Menschen bei uns die viel...",9,16,Gerechtigkeit ist die zentrale Voraussetzung f...,- Unser Wohlstand und unsere Lebensqualität hä...,0.612255,0.579072,42.730321,190.189781
17,Unser gemeinsames Ziel ist Vollbeschäftigung i...,680,53,Die universelle Geltung und die Unteilbarkeit ...,- Wir haben in dieser Wahlperiode unsere Inves...,0.647968,0.658968,3228.513120,630.003650
18,"Wir wollen unser Land erneuern, in die Zukunft...",285,53,- Die besten Schulen. Mit unserem Schulmoderni...,- Wir haben in dieser Wahlperiode unsere Inves...,0.492994,0.509692,1353.126822,630.003650
19,Wir investieren in unser Land. Wir sorgen für ...,285,16,- Die besten Schulen. Mit unserem Schulmoderni...,- Unser Wohlstand und unsere Lebensqualität hä...,0.561945,0.523114,1353.126822,190.189781


# Plot-Darstellung

In [174]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool, PanTool, ResetTool, WheelZoomTool, LabelSet, Label

hover=HoverTool(
    tooltips=[
        ( 'KV',   '@K'  ),
        ( 'SPD',  '@S'  ), # use @{ } for field names with spaces
        ( 'Union', '@U' ),
    ],
    mode='hline'
)
def strCutLen(data,maxlen=150):
    return (data[:maxlen] + '..') if len(data) > maxlen else data

source = ColumnDataSource(data=dict(
    K=Vergleich.K.map(lambda x:strCutLen(x)),
    S=Vergleich.S.map(lambda x:strCutLen(x)),
    U=Vergleich.U.map(lambda x:strCutLen(x)),
))


tools = [hover, WheelZoomTool(), PanTool(), ResetTool()] 
p1 = figure(tools=tools,plot_height=800, plot_width=1000,x_range=(-1.2,1.2 ), y_range=(KS.index.max()*1.1,-200))
p1.segment(-1, Vergleich.yS, 0, KS.index,color="red", line_width=Vergleich.sKS)
p1.segment(1, Vergleich.yU, 0, KU.index,color="blue", line_width=Vergleich.sKU)
p1.line(0,KU.index,source=source)

Labelsource = ColumnDataSource(data=dict(x=[-1.05,-0.03,0.9],
                                    y=[0,0,0],
                                    names=['SPD','KV','CDU/CSU']))
labels = LabelSet(x='x', y='y', text='names', level='glyph',
              x_offset=0, y_offset=0, source=Labelsource, render_mode='canvas')

p1.add_layout(labels)

output_notebook()

show(p1)

Loading BokehJS ...

In [155]:
Vergleich.sKS.sum()

738.5059819602895

In [175]:
Vergleich.sKU.sum()

740.0822295201051

In [181]:
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return Markdown(df_formatted.to_csv(sep="|", index=False))

MD=pandas_df_to_markdown_table(Vergleich[["sKS","KS","S","K","U","KU","sKU"]])


In [184]:
try:
    writer = open("vergleich.md","w")
except:
    print("Could not open file ")
    quit() 

writer.write(MD.data)
writer.close()